In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import DeepFool
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 03:30:51.405237: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 03:30:51.408329: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 03:30:51.441779: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 03:30:51.441833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 03:30:51.443914: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
x_train = x_train.values
y_train = y_train.values
x_val = x_val.values
y_val = y_val.values
x_test = x_test.values
y_test = y_test.values

In [5]:
model = joblib.load("/home/jovyan/Edge-IIoT/dl/dl_new/dnn.joblib")

2024-02-26 03:30:57.415064: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://eac59fe9c2164499ab273235aca2db50: INVALID_ARGUMENT: ram://eac59fe9c2164499ab273235aca2db50 is a directory.


In [6]:
x_test_trimmed = x_test[20700:21200]
y_test_trimmed = y_test[20700:21200]

In [7]:
np.unique(y_test_trimmed)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [8]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=15,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [9]:
attack = DeepFool(classifier=classifier, epsilon=0.01)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   0%|          | 0/500 [00:00<?, ?it/s]

DeepFool: 100%|██████████| 500/500 [38:45<00:00,  4.65s/it]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 971us/step
Accuracy: 0.268
AUC: 0.5770002934896251

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604


In [11]:
attack = DeepFool(classifier=classifier, epsilon=0.05)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 500/500 [40:24<00:00,  4.85s/it]


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 832us/step
Accuracy: 0.268
AUC: 0.5770002934896251

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604


In [42]:
attack = DeepFool(classifier=classifier, epsilon=0.1)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool:   9%|▉         | 45/500 [04:56<49:46,  6.56s/it]

DeepFool: 100%|██████████| 500/500 [55:22<00:00,  6.65s/it]


In [43]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 781us/step
Accuracy: 0.268
AUC: 0.5770860080505064

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604


In [44]:
attack = DeepFool(classifier=classifier, epsilon=0.15)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 500/500 [56:00<00:00,  6.72s/it]


In [45]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 765us/step
Accuracy: 0.268
AUC: 0.5770860080505064

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604


In [46]:
attack = DeepFool(classifier=classifier, epsilon=0.2)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 500/500 [56:02<00:00,  6.73s/it]


In [47]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 764us/step
Accuracy: 0.268
AUC: 0.5770860080505064

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604


In [48]:
attack = DeepFool(classifier=classifier, epsilon=0.25)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 500/500 [56:19<00:00,  6.76s/it]


In [49]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 735us/step
Accuracy: 0.268
AUC: 0.5770860080505064

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604


In [50]:
attack = DeepFool(classifier=classifier, epsilon=0.3)

x_test_adv = attack.generate(x=x_test_trimmed)

DeepFool: 100%|██████████| 500/500 [56:20<00:00,  6.76s/it]


In [51]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

16/16 [==============================] - 0s 745us/step
Accuracy: 0.268
AUC: 0.5770860080505064

macro
Precision: 0.8010491469315
Recall: 0.21143775100401604
F1 Score: 0.18227665093336734

weighted
Precision: 0.7242594346829642
Recall: 0.268
F1 Score: 0.1862867712718459

Mean FNR: 0.7885622489959839
Mean TNR: 0.942562835975234
Mean FPR: 0.057437164024765945
Mean TPR: 0.21143775100401604
